# Install WRDS

In [ ]:
# pip install wrds

In [1]:
import pandas as pd
import numpy as np

# Extract S&P CIK codes

In [2]:
sp_500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp_500
cik = []
for i in range(len(sp_500["CIK"])):
    temp = str(sp_500["CIK"][i])
    while len(temp) != 10:
        temp = "0" + temp
    cik.append(temp)
sp_500["CIK"] = pd.Series(cik)

# Import WRDS and connect to web (enter your credentials when prompted)

In [5]:
import wrds
db = wrds.Connection()

Enter your WRDS username [Black]:grivoire
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


# Look at all the tables available under ciq library

In [6]:
db.list_tables("ciq")

['chars',
 'ciqaddress',
 'ciqaddresstype',
 'ciqadvisortype',
 'ciqbusinessdescription',
 'ciqbusinessdescriptionlong',
 'ciqcapstdtasrptdclasstype',
 'ciqcapstdtclasstype',
 'ciqcapstdtcompntasrptddata',
 'ciqcapstdtcomponent',
 'ciqcapstdtconvertibletype',
 'ciqcapstdtcumulativetype',
 'ciqcapstdtdescription',
 'ciqcapstdtintbenchmarktype',
 'ciqcapstdtinterestrate',
 'ciqcapstdtintratetype',
 'ciqcapstdtleveltype',
 'ciqcapstdtnonrecoursetype',
 'ciqcapstdtparticipatingtype',
 'ciqcapstdtredeemabletype',
 'ciqcapstdtsecuredtype',
 'ciqcapstdtsubtype',
 'ciqcapstdttype',
 'ciqcapsteqauthrzdsharestype',
 'ciqcapsteqcomponent',
 'ciqcapsteqcomponentdata',
 'ciqcapsteqconvertibletype',
 'ciqcapsteqsubtype',
 'ciqcapsteqtype',
 'ciqcapsteqvotingrightstype',
 'ciqchartype',
 'ciqchartypetosubtype',
 'ciqcommittee',
 'ciqcompany',
 'ciqcompanyindustrytree',
 'ciqcompanyrel',
 'ciqcompanyreltype',
 'ciqcompanystatustype',
 'ciqcompanytype',
 'ciqcompensation',
 'ciqcompensationadjustment',

# Access the table that links CIK to CIQ IDs

In [7]:
db.describe_table(library = "ciq", table = "wrds_cik")

Approximately 373733 rows in ciq.wrds_cik.


,name,nullable,type,comment
0,companyid,True,DOUBLE_PRECISION,None
1,cik,True,VARCHAR(10),None
2,startdate,True,DATE,None
3,enddate,True,DATE,None
4,companyname,True,VARCHAR(400),None


In [8]:
CIK = tuple(sp_500['CIK'])

In [9]:
sql_query = 'SELECT * from ciq.wrds_cik WHERE cik IN'
sql_query += f'{CIK}'
sql_query
# Run query,  result in Pandas dataframe format
data = db.raw_sql(sql_query)
data

,companyid,cik,startdate,enddate,companyname
0,2.474830e+05,0000001800,None,None,Abbott Laboratories
1,1.688640e+05,0000002488,None,None,"Advanced Micro Devices, Inc."
2,2.483560e+05,0000002969,None,None,"Air Products and Chemicals, Inc."
3,2.494890e+05,0000004127,None,2002-06-25,"Alpha Industries, Inc."
4,1.462309e+06,0000004127,2002-06-26,None,"Skyworks Solutions, Inc."
...,...,...,...,...,...
520,6.312364e+08,0001792044,None,2020-11-10,Upjohn Inc.
521,6.537486e+08,0001821825,None,None,Organon & Co.
522,1.698380e+05,0001841666,None,None,APA Corporation
523,3.136719e+06,0001868275,None,None,Constellation Energy Corporation


## Remove duplicated, legacy CIK entries

In [10]:
CIQ = data[data["enddate"].isna()]
CIQ

,companyid,cik,startdate,enddate,companyname
0,2.474830e+05,0000001800,None,None,Abbott Laboratories
1,1.688640e+05,0000002488,None,None,"Advanced Micro Devices, Inc."
2,2.483560e+05,0000002969,None,None,"Air Products and Chemicals, Inc."
4,1.462309e+06,0000004127,2002-06-26,None,"Skyworks Solutions, Inc."
6,4.162645e+08,0000004281,2016-10-28,None,Howmet Aerospace Inc.
...,...,...,...,...,...
519,2.902030e+05,0001792044,2020-11-11,None,Viatris Inc.
521,6.537486e+08,0001821825,None,None,Organon & Co.
522,1.698380e+05,0001841666,None,None,APA Corporation
523,3.136719e+06,0001868275,None,None,Constellation Energy Corporation


In [26]:
IDs = sp_500.set_index('CIK').join(CIQ.set_index('cik'))
IDs = IDs.rename_axis('CIK', axis='index').reset_index()
IDs['companyid'] = IDs['companyid'].astype(int)
IDs

,CIK,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,Founded,companyid,startdate,enddate,companyname
0,0000001800,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1888,247483,None,None,Abbott Laboratories
1,0000002488,AMD,AMD,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,1969,168864,None,None,"Advanced Micro Devices, Inc."
2,0000002969,APD,Air Products and Chemicals,Materials,Industrial Gases,"Allentown, Pennsylvania",1985-04-30,1940,248356,None,None,"Air Products and Chemicals, Inc."
3,0000004127,SWKS,Skyworks Solutions,Information Technology,Semiconductors,"Irvine, California",2015-03-12,2002,1462309,2002-06-26,None,"Skyworks Solutions, Inc."
4,0000004281,HWM,Howmet Aerospace,Industrials,Aerospace & Defense,"Pittsburgh, Pennsylvania",1957-03-04,2016,416264458,2016-10-28,None,Howmet Aerospace Inc.
...,...,...,...,...,...,...,...,...,...,...,...,...
498,0001792044,VTRS,Viatris,Health Care,Pharmaceuticals,"Pittsburgh, Pennsylvania",2004-04-23,1961,290203,2020-11-11,None,Viatris Inc.
499,0001821825,OGN,Organon & Co.,Health Care,Pharmaceuticals,"Jersey City, New Jersey",2021-06-03,2021,653748561,None,None,Organon & Co.
500,0001841666,APA,APA Corporation,Energy,Oil & Gas Exploration & Production,"Houston, Texas",1997-07-28,1954,169838,None,None,APA Corporation
501,0001868275,CEG,Constellation Energy,Utilities,Multi-Utilities,"Baltimore, Maryland",2022-02-02,1999,3136719,None,None,Constellation Energy Corporation


# Access the tables that contain earnings transcript for given CIQs

In [13]:
db.describe_table(library = "ciq", table = "wrds_transcript_detail")

Approximately 1460372 rows in ciq.wrds_transcript_detail.


,name,nullable,type,comment
0,companyid,True,DOUBLE_PRECISION,None
1,keydevid,True,DOUBLE_PRECISION,None
2,transcriptid,True,DOUBLE_PRECISION,None
3,headline,True,VARCHAR(381),None
4,mostimportantdateutc,True,DATE,None
5,mostimportanttimeutc,True,DOUBLE_PRECISION,None
6,keydeveventtypeid,True,DOUBLE_PRECISION,None
7,keydeveventtypename,True,VARCHAR(400),None
8,companyname,True,VARCHAR(400),None
9,transcriptcollectiontypeid,True,DOUBLE_PRECISION,None


In [14]:
db.describe_table(library = "ciq", table = "wrds_transcript_person")

Approximately 76100952 rows in ciq.wrds_transcript_person.


,name,nullable,type,comment
0,transcriptid,True,DOUBLE_PRECISION,None
1,transcriptcomponentid,True,DOUBLE_PRECISION,None
2,componentorder,True,DOUBLE_PRECISION,None
3,transcriptcomponenttypeid,True,DOUBLE_PRECISION,None
4,transcriptcomponenttypename,True,VARCHAR(400),None
5,transcriptpersonid,True,DOUBLE_PRECISION,None
6,transcriptpersonname,True,VARCHAR(800),None
7,proid,True,DOUBLE_PRECISION,None
8,companyofperson,True,VARCHAR(800),None
9,speakertypeid,True,DOUBLE_PRECISION,None


In [15]:
db.describe_table(library = "ciq", table = "ciqtranscriptcomponent")

Approximately 76517816 rows in ciq.ciqtranscriptcomponent.


,name,nullable,type,comment
0,transcriptcomponentid,True,INTEGER,None
1,transcriptid,True,INTEGER,None
2,componentorder,True,SMALLINT,None
3,transcriptcomponenttypeid,True,SMALLINT,None
4,transcriptpersonid,True,INTEGER,None
5,componenttext,True,VARCHAR,None


In [16]:
CIQ_list = tuple(CIQ['companyid'])

In [17]:
def WRDSpuller(year, CIQ = CIQ_list):
    # Create a sql query string
    sql_query = '''SELECT a.companyid, a.transcriptid, a.headline, a.mostimportantdateutc, a.companyname, 
    b.word_count, c.componenttext FROM'''
    sql_query += f'(SELECT * FROM ciq.wrds_transcript_detail WHERE companyid in {CIQ} '
    sql_query += ''' and date_part('year',mostimportantdateutc)='''
    sql_query += f'{year}'
    sql_query += ''' and keydeveventtypename = 'Earnings Calls')  as a, ciq.wrds_transcript_person as b, ciq.ciqtranscriptcomponent as c WHERE a.transcriptid=b.transcriptid and b.transcriptcomponentid=c.transcriptcomponentid
    ORDER by a.transcriptid, b.componentorder'''
    # Run query,  result in Pandas dataframe format
    transcripts = db.raw_sql(sql_query)
    transcripts.to_pickle(f'S&P{year}.pkl')
    return transcripts

In [ ]:
for i in range(2006, 2024):
    year_variable = "WRDSpuller_" + str(i)
    exec(year_variable + " = WRDSpuller(i)")
    print("Successfully pulled out for year", i)

In [18]:
IDs[IDs["CIK"] == "0001652044"]

,CIK,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,Founded,companyid,startdate,enddate,companyname
472,0001652044,GOOGL,Alphabet Inc. (Class A),Communication Services,Interactive Media & Services,"Mountain View, California",2014-04-03,1998,29096,None,None,Alphabet Inc.
473,0001652044,GOOG,Alphabet Inc. (Class C),Communication Services,Interactive Media & Services,"Mountain View, California",2006-04-03,1998,29096,None,None,Alphabet Inc.


In [23]:
import pandas as pd

def concatenator(data, ID):
    summary = []
    for i in range(len(data["headline"].unique())):
        focus_headline = data[data["headline"] == data["headline"].unique()[i]].dropna()
        companyid_x, companyname_x, mostimportantdateutc_x = focus_headline["companyid"].iloc[0], focus_headline["companyname"].iloc[0], focus_headline["mostimportantdateutc"].iloc[0]
        content = str()
        word_counts = 0
        for x in range(len(focus_headline["componenttext"])): 
            content += focus_headline["componenttext"].iloc[x] + " "
            word_counts += focus_headline["word_count"].iloc[x]
        summary.append([companyid_x, companyname_x, mostimportantdateutc_x, word_counts, content[:-1]])

    table = pd.DataFrame(summary)
    table.columns = ["companyid", "companyname", "mostimportantdateutc", "word_counts", "content"]
    table.companyid = table.companyid.astype(int)
    
    # Set 'companyid' as the index for both dataframes
    ID_table = ID.copy()
    ID_table.set_index('companyid', inplace=True)
    table.set_index('companyid', inplace=True)

    # Join 'table' and 'ID' dataframes
    table = ID_table[['CIK','Symbol','Security','GICS Sector','GICS Sub-Industry','Headquarters Location']].join(table, how='inner')

    # Reset the index
    table.reset_index(inplace=True)

    table = table.sort_values(by=['companyname','mostimportantdateutc']).reset_index(drop=True)
    return table


In [31]:
for i in range(2006, 2024):
    data = pd.read_pickle(f"S&P{i}.pkl")
    proc = concatenator(data,IDs)
    proc.to_pickle(f'Clean_S&P{i}.pkl')
    print(f"Successfully Cleaned & Pickled S&P{i}")

Successfully Cleaned & Pickled S&P2006
Successfully Cleaned & Pickled S&P2007
Successfully Cleaned & Pickled S&P2008
Successfully Cleaned & Pickled S&P2009
Successfully Cleaned & Pickled S&P2010
Successfully Cleaned & Pickled S&P2011
Successfully Cleaned & Pickled S&P2012
Successfully Cleaned & Pickled S&P2013
Successfully Cleaned & Pickled S&P2014
Successfully Cleaned & Pickled S&P2015
Successfully Cleaned & Pickled S&P2016
Successfully Cleaned & Pickled S&P2017
Successfully Cleaned & Pickled S&P2018
Successfully Cleaned & Pickled S&P2019
Successfully Cleaned & Pickled S&P2020
Successfully Cleaned & Pickled S&P2021
Successfully Cleaned & Pickled S&P2022
Successfully Cleaned & Pickled S&P2023
